In [1]:
import pandas as pd
import numpy as np
import gzip
import os
import datetime
import pandas as pd
import calendar
import numpy as np
import glob
import sys
import gc
import re
#import quantstats as qs
import time

import matplotlib.pyplot as plt

import nest_asyncio # install this package to avoid running in loop

import urllib
import pandas_ta as ta

import warnings

import requests

import hmac
import hashlib

import aiohttp
import asyncio

import logging
from datetime import timezone

import nest_asyncio

# Filter depreciation warnings from pandas regarding the append method
warnings.filterwarnings('ignore', category=FutureWarning, message=".*append.*")
pd.options.mode.chained_assignment = None

# Set pandas to display all digits for floating-point numbers
pd.options.display.float_format = '{:.8f}'.format

import os
from dotenv import load_dotenv

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Ignore only FutureWarning
warnings.filterwarnings('ignore', category=FutureWarning)

# Ignore only RuntimeWarning
warnings.filterwarnings('ignore', category=RuntimeWarning)

import importlib


from Orats_options_fetch_functions import *

from faster_numba_strategy_functions import *

import strategy_functions

importlib.reload(strategy_functions)
from strategy_functions import *

# import Common_functions

# importlib.reload(strategy_functions)
# importlib.reload(Common_functions)

# from Common_functions import *
# coin_desk_api_key =  "e43c1fd991e660b9bf959645f0800bb7e76fb4a3537ab773cec62b2fad31af2d"
from datetime import time
from pathlib import Path
# Get the user's home directory dynamically
home_dir = r"C:\Data\Options_logs"
#====================================

strategy_name = "Drawdown_hedge"

# Create a folder for the strategy_name if it doesn't exist in home directory

strategy_folder = os.path.join(home_dir, strategy_name)
if not os.path.exists(strategy_folder):
    os.makedirs(strategy_folder)

# Create the full path to the output file. this file contains all the trades

output_all_trades_file = os.path.join(strategy_folder, f"All_trades_{strategy_name}.csv")
trade_html_file = os.path.join(strategy_folder, f"Trade_stats_{strategy_name}.html")

#=== location to store the trade dataframe for debug

base_folder_df = "strategy_dataframe"
df_folder = os.path.join(strategy_folder, base_folder_df)
if not os.path.exists(df_folder):
    os.makedirs(df_folder)
    print("\n Created strategy dataframe folder : " ,df_folder )

#----- symbol_wise_trades

trades_folder = "sym_wise_trades"
trades_strategy_folder = os.path.join(strategy_folder, trades_folder)

if not os.path.exists(trades_strategy_folder):
    print("\n Making folder for symbol wise trades of strategy" , trades_strategy_folder)
    os.makedirs(trades_strategy_folder)

logger = setup_logger("Straddle_0DTE", strategy_folder, log_file="strategy.log" , to_console=False)


In [2]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt

# Connect to an in-memory database
con = duckdb.connect()

# --- Your setup code (which is correct) ---
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")
con.execute("""
SET s3_endpoint='122.176.133.73:9000';
SET s3_use_ssl=false;
SET s3_access_key_id='minioadmin';
SET s3_secret_access_key='minioadmin';
""")
con.execute("SET s3_url_style='path';") 
con.execute("PRAGMA threads=8;")
con.execute("PRAGMA enable_object_cache;")

# --- THE MISSING STEP: Querying a file from S3 ---
# Replace with your actual S3 bucket and file path
# This can be a .parquet, .csv, or .json file
s3_file_path = 's3://duckdata/ORATS/Options/**/*.parquet'

try:
    # Execute a query on the S3 file and fetch the result as a Pandas DataFrame
    print(f"Querying data from: {s3_file_path}")
    df = con.execute(f"""
        SELECT *
        FROM '{s3_file_path}'
        LIMIT 2;
    """).df()

    print("\n✅ Query successful! First 5 rows of data:")
    print(df.head())

except Exception as e:
    print(f"\n❌ An error occurred during the query: {e}")

# finally:
#     # It's good practice to close the connection
#     con.close()

Querying data from: s3://duckdata/ORATS/Options/**/*.parquet

✅ Query successful! First 5 rows of data:
      expiry       strike  dte  optionType                  ts         low  \
0 2024-01-26 421.00000000   25          -1 2024-01-02 17:13:00 17.18000000   
1 2024-01-12 414.00000000   11          -1 2024-01-02 17:13:00 10.41500000   

         high        open       close     volume  ...      askSize  \
0 17.18000000 17.18000000 17.18000000 0.00000000  ...  50.00000000   
1 10.41500000 10.41500000 10.41500000 0.00000000  ... 151.00000000   

     bidPrice    askPrice      bidIv      askIv         iv          oi  \
0 17.12000000 17.24000000 0.13785700 0.14511500 0.14148600 15.00000000   
1 10.34000000 10.49000000 0.14267100 0.15199600 0.14733300 54.00000000   

    stockPrice        day ticker  
0 404.00000000 2024-01-02    QQQ  
1 404.00000000 2024-01-02    QQQ  

[2 rows x 21 columns]


In [3]:
dd_fol = r"C:\Users\aakas\Nasdaq_Live_Strategy\sym_wise_trades\system_drawdown.csv"

ddf = pd.read_csv(dd_fol)
ddf['timestamp'] = pd.to_datetime(ddf['timestamp'])
# 1. Calculate the difference in 'Total_Drawdown' from the previous row
drawdown_change = ddf['Total_Drawdown'].diff()

# 2. Divide the change by 250 and convert to integer to get the multiples
# .fillna(0) handles the first row, which has no previous value
ddf['hedge'] = (drawdown_change.fillna(0) / 250).astype(int)
ddf.set_index('timestamp' , inplace=True)
ddf = ddf[ddf.index.year>=2024]

In [4]:
ddf

,Total_Drawdown,hedge
timestamp,,
2024-01-18 11:00:00,-2366.82694337,1
2024-01-18 11:15:00,-1437.76985629,3
2024-01-18 11:30:00,-3153.56312648,-6
2024-01-18 11:45:00,-3199.34064174,0
2024-01-18 12:00:00,-8014.72034787,-19
...,...,...
2024-09-06 15:00:00,-915.40280342,-1
2024-09-06 15:15:00,-998.62983322,0
2024-09-06 15:30:00,-371.48838043,2


In [5]:


ifol = r"C:\Data\Aggregate_Data\Options\USA\Raw_SPY_1min_2024_2025_ORATS_close.parquet"
#== read the RELIANCE data

com_df2 = pd.read_parquet(ifol)

if 'timestamp' in com_df2.columns:

    com_df2['timestamp'] = pd.to_datetime(com_df2['timestamp'])
    com_df2.set_index('timestamp' , inplace=True)


com_df2.dropna(how='all', inplace=True)

# vix_df = pd.read_parquet(r"C:\Data\Aggregate_Data\VIX_1_min_2019_2025.parquet")
# com_df2 = com_df2.join(vix_df)


final_symbols = com_df2.filter(like='close').columns.str.replace('_close', '')
print("Total symbols found after all filtering: ", len(final_symbols))

# Calculate the number of rows in the DataFrame

com_df2[[f"{symbol}_intraday_low" for symbol in final_symbols]] =com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: intraday_low(com_df2, low_col=x.name,agg_func='min'))

com_df2[[f"{symbol}_intraday_high" for symbol in final_symbols]] =com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: intraday_high(com_df2, high_col=x.name,agg_func='max'))

com_df2[[f"{symbol}_todayo" for symbol in final_symbols]] = com_df2[[f'{symbol}_close' for symbol in final_symbols]].resample('D').transform('first')

# com_df2[[f"{symbol}_daily_ROC" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].pct_change(periods=50)

# com_df2[[f"{symbol}_ROC" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].pct_change()

# com_df2[[f"{symbol}_ROC_4" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].pct_change(periods=4)

com_df2[[f"{symbol}_prevdayl" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: get_x_day_low_numba(com_df2, n=1, column=x.name))

com_df2[[f"{symbol}_prevdayh" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: get_x_day_high_numba(com_df2, n=1, column=x.name))

com_df2[[f"{symbol}_secdayh" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: get_x_day_high_numba(com_df2, n=2, column=x.name))


# This single call does it for the 20-day MA
ma_20d_df = calculate_daily_ma_vectorized(df=com_df2, symbols=final_symbols, periods=3 , agg_func='last' , col_suffix='close')

    # Join the new features back to the main DataFrame
com_df2 = pd.concat([com_df2, ma_20d_df] , axis=1 , join='outer')


com_df2[[f"{symbol}_ma" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].rolling(2).mean()

com_df2[[f"{symbol}_ROC_30" for symbol in final_symbols]] = com_df2[[f"{symbol}_ma" for symbol in final_symbols]].pct_change(periods=30)


#com_df2[[f"{symbol}_ma_3d_low" for symbol in final_symbols]] = com_df2[[f"{symbol}_ma" for symbol in final_symbols]].values / com_df2[[f"{symbol}_3d_MA" for symbol in final_symbols]].values

com_df2[[f"{symbol}_prevdayc" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: calculate_prev_day_close(com_df2, close_col=x.name, n_days=1))

com_df2[[f"{symbol}_secdayc" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: calculate_prev_day_close(com_df2, close_col=x.name, n_days=2))

# #=== hourly volume
# com_df2[[f"{symbol}_vol_ma" for symbol in final_symbols]] = com_df2[[f"{symbol}_volume" for symbol in final_symbols]].rolling(4).mean()

# #==avg 3 day volume

# com_df2[[f"{symbol}_3d_vol_MA" for symbol in final_symbols]] = com_df2[[f"{symbol}_volume" for symbol in final_symbols]].apply(lambda x: daily_moving_average(com_df2, timeframe='1D', column=x.name, periods=3, agg_func='last'))

com_df2[[f"{symbol}_ma2_low" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].rolling(5).mean()


com_df2[[f"{symbol}_3dlow" for symbol in final_symbols]] = com_df2[[f"{symbol}_ma2_low" for symbol in final_symbols]].apply(lambda x: get_x_day_low_numba(com_df2, n=3, column=x.name))

com_df2[[f"{symbol}_3dhigh" for symbol in final_symbols]] = com_df2[[f"{symbol}_ma2_low" for symbol in final_symbols]].apply(lambda x: get_x_day_high_numba(com_df2, n=3, column=x.name))



# 1. Get the lists of column names
high_3d_cols = [f"{symbol}_3dhigh" for symbol in final_symbols]
low_3d_cols = [f"{symbol}_3dlow" for symbol in final_symbols]

# 2. Perform the calculation on NumPy arrays
range_values = (com_df2[high_3d_cols].values / com_df2[low_3d_cols].values) - 1

# 3. Create a new DataFrame from the results
# This is the key step to avoid the error.
range_cols = [f"{symbol}_3d_rng" for symbol in final_symbols]
range_df = pd.DataFrame(range_values, index=com_df2.index, columns=range_cols)

# 4. Join the new DataFrame back to the main one
com_df2 = com_df2.join(range_df)



# # Calculate highest high of last 4 bars
# com_df2[[f"{symbol}_high_4bar" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].rolling(10).max()

# # Calculate lowest low of last 4 bars
# com_df2[[f"{symbol}_low_4bar" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].rolling(10).min()

# # 1. Get the lists of column names
# high_cols = [f"{symbol}_high_4bar" for symbol in final_symbols]
# low_cols = [f"{symbol}_low_4bar" for symbol in final_symbols]

# # 2. Perform the calculation on the NumPy arrays
# range_values = com_df2[high_cols].values - com_df2[low_cols].values

# # 3. Create a new DataFrame from the result (This is the key step)
# range_cols = [f"{symbol}_range_4bar" for symbol in final_symbols]
# range_df = pd.DataFrame(range_values, index=com_df2.index, columns=range_cols)

# # 4. Join the new DataFrame back to the main one
# com_df2 = com_df2.join(range_df)


# # Calculate mean range (average of the 4-bar ranges)
# com_df2[[f"{symbol}_mean_range_4bar" for symbol in final_symbols]] = com_df2[[f"{symbol}_range_4bar" for symbol in final_symbols]].rolling(12).mean()


# gc.collect()

# for symbol in final_symbols:

   
#     com_df2[f'{symbol}_iATR2'] = com_df2[f"{symbol}_ATR"].values / com_df2[f"{symbol}_close"].values


# 1. Define the column groups
close_cols = [f"{symbol}_close" for symbol in final_symbols]
low_3d_cols = [f"{symbol}_3dlow" for symbol in final_symbols]
high_3d_cols = [f"{symbol}_3dhigh" for symbol in final_symbols]

# 2. Perform the calculation in a more chained and memory-conscious way
# Using .to_numpy(dtype=np.float32) can cut memory usage by 50%
numerator = com_df2[close_cols].to_numpy(dtype=np.float32) - com_df2[low_3d_cols].to_numpy(dtype=np.float32)
denominator = com_df2[high_3d_cols].to_numpy(dtype=np.float32) - com_df2[low_3d_cols].to_numpy(dtype=np.float32)

# np.divide handles division by zero safely
percentile_values = np.divide(numerator, denominator, where=denominator != 0)

# 3. Create the new feature DataFrame
pctl_cols = [f"{symbol}_3d_Pctl" for symbol in final_symbols]
percentile_df = pd.DataFrame(percentile_values, index=com_df2.index, columns=pctl_cols)

# 4. Clip values and join back (same as your code)
percentile_df = percentile_df.clip(0, 1)
com_df2 = com_df2.join(percentile_df)

com_df2['decent'] = np.where((com_df2['SPY_todayo'] < com_df2['SPY_prevdayc'] * 1.005) & (com_df2['SPY_todayo'] > com_df2['SPY_prevdayc'] * 0.98), 1, 0)
com_df = com_df2[com_df2['decent']==1]
com_df = com_df[com_df.index.year>=2024]

Total symbols found after all filtering:  1
Calculating 3-day MA for 1 symbols...


In [6]:
com_df = ddf.join(com_df2)

In [7]:

def fetch_bulk_option_data_call_put(con, ticker, end_date_str, n_days, strikes: List[float], expiries: List[str] , option_type: int = 0 ):
    
    """
    Fetches a range of minute-level data for MULTIPLE option contracts
    in a single, efficient query.

    Args:
        con: The database connection object.
        ticker (str): The stock ticker (e.g., 'SPY').
        end_date_str (str): The end date for the data fetch in 'YYYY-MM-DD' format.
        n_days (int): The number of business days to look back.
        strikes (List[float]): A list of strike prices to fetch.
        expiries (List[str]): A list of expiry dates in 'YYYY-MM-DD' format.
        option_type (int, optional): The type of option to fetch. 
                                     1 for Calls, -1 for Puts, 0 for Both. 
                                     Defaults to 0.
    
    Returns:
        pd.DataFrame: A DataFrame containing the requested option data, or an empty
                      DataFrame if no data is found or an error occurs.
    """

    # 1. Get the list of business days to query (same as before)
    nyse = mcal.get_calendar('NYSE')
    start_buffer = pd.to_datetime(end_date_str) - pd.Timedelta(days=n_days)
    schedule = nyse.schedule(start_date=start_buffer, end_date=end_date_str)
    business_day_list = [d.strftime('%Y-%m-%d') for d in schedule.index[-n_days:]]
    
    # 1. Validate the option_type parameter first
    if option_type not in [1, -1, 0]:
        print(f"❌ Bad input for option_type: '{option_type}'. Must be 1 (Call), -1 (Put), or 0 (Both).")
        return pd.DataFrame()


    # 2. Build the list of S3 paths (same as before)
    path_list = [
        f"'s3://duckdata/ORATS/Options/ticker={ticker}/day={d}/*.parquet'"
        for d in business_day_list ]
    
    # ▼▼▼ NEW QUERY LOGIC ▼▼▼
    # 3. Format the lists for the SQL 'IN' clause
    strikes_str = ",".join(map(str, strikes)) # For numbers: 470.0,471.0,472.0
    expiries_str = ",".join([f"'{e}'" for e in expiries]) # For strings: '2024-01-05','2024-01-08'
    
    # ▼▼▼ NEW LOGIC: DYNAMIC QUERY FILTER ▼▼▼
    # 2. Build the optionType filter string based on the parameter
    option_type_filter_str = ""
    if option_type == 1:
        option_type_filter_str = "AND optionType = 1"
    elif option_type == -1:
        option_type_filter_str = "AND optionType = -1"



    query = f"""
        SELECT ts, strike, expiry, close, bidPrice, askPrice, volume, oi, dte, optionType , iv
        FROM read_parquet([{",".join(path_list)}])
        WHERE 
            CAST(strike AS FLOAT) IN ({strikes_str}) AND
            expiry IN ({expiries_str})
            {option_type_filter_str} 
        ORDER BY ts;
    """
    
    try:
        bulk_df = con.execute(query).df()
        
        if bulk_df.empty:
            return pd.DataFrame()
            
        # Perform timezone conversion and set index (same as before)
        bulk_df['ts'] = pd.to_datetime(bulk_df['ts'].dt.tz_localize('UTC').dt.tz_convert('America/New_York').dt.tz_localize(None))
        bulk_df.set_index('ts' , inplace=True)
        return bulk_df
        
    except Exception as e:
        print(f"An error occurred during bulk fetch: {e}")
        return pd.DataFrame()

In [8]:

# This defines the value of one strike step (e.g., for SPY, strikes are 1 point apart)
strike_interval = 1.0

# Example 1: 0DTE At-the-Money LONG STRADDLE
strategy_config = {
    'Leg1': {'action': 'SELL', 'option_type': 1, 'expiry_offset': 0, 'strike_offset': 1},  # Sell ATM Call
    }   # Buy 1DTE call



In [9]:
def fetch_atm_option_at_timestamp(con, ticker: str, timestamp: pd.Timestamp, option_type: int):
    """
    Fetches specific data for the single, most At-The-Money (ATM) option contract 
    with DTE=1 at a specific timestamp.

    Args:
        con: The DuckDB connection object.
        ticker (str): The stock ticker (e.g., 'SPY').
        timestamp (pd.Timestamp): The specific timestamp to query (assumed to be in UTC).
        option_type (int): The option type (1 for Calls, -1 for Puts).

    Returns:
        pd.DataFrame: A DataFrame with a single row containing the requested columns.
    """
    try:
        # Format the UTC timestamp for the query
        utc_timestamp_str = timestamp.strftime('%Y-%m-%d %H:%M:%S')
        date_str = timestamp.strftime('%Y-%m-%d')

        query = f"""
            WITH RankedOptions AS (
                SELECT ts, strike, expiry, close, bidPrice, askPrice, volume, oi, dte, optionType , iv
                    ROW_NUMBER() OVER(
                        ORDER BY ABS(strike - stockPrice) ASC
                    ) as atm_rank
                FROM read_parquet('s3://duckdata/ORATS/Options/ticker={ticker}/day={date_str}/*.parquet')
                WHERE 
                    ts = '{utc_timestamp_str}' AND
                    dte = 1 AND
                    optionType = {option_type}
            )
            -- VVV THIS LINE IS CHANGED VVV --
            SELECT 
                ts, strike, expiry, stockPrice, ticker
            FROM RankedOptions
            WHERE atm_rank = 1;
        """
        
        option_data = con.execute(query).df()
        return option_data

    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

In [10]:
# ==============================================================================
# SCRIPT STARTS HERE - SIMPLIFIED iloc LOOP
# ==============================================================================

# Initialize state dictionaries ONCE before the loop
open_positions = {} # Tracks details of open positions
final_trade_log_list = []
option_multiplier = 100

print(f"🚀 Starting backtest for {len(com_df)} timestamps...")

# --- Main Loop: Iterate Through Each Row using an integer index ---
for i in range(len(com_df))[1:]:
    # Get the timestamp and row data using the integer index 'i'
    timestamp = com_df.index[i]
    row = com_df.iloc[i]

    # --- Part A: Manage Exits for OPEN positions ---
    for leg_name, position_details in list(open_positions.items()):
        # EOD Exit: Close all positions
        if timestamp.time() >= time(15, 45):
            current_option_data = fetch_single_option_at_timestamp(
                con, 'SPY', timestamp,
                strike=position_details['strike'],
                expiry_str=position_details['expiry'],
                option_type=1,
                DTE=1
            )

            if current_option_data is not None and  len(current_option_data)>=1 :
                exit_price = current_option_data['bidPrice'].iloc[0] # Buy back to close
                profit = (position_details['entry_price'] - exit_price) * position_details['qty']


                print(f"Exit called at {timestamp} and entry was {position_details['entry_price']} and exit is {exit_price}")

                # --- ADD THIS SECTION TO LOG THE TRADE ---
                trade_details = {
                    'name': leg_name,
                    'Side': 'SELL', # This is a short call hedge
                    'entry_date': position_details['entry_timestamp'],
                    'entry_price': position_details['entry_price'],
                    'qty': position_details['qty'],
                    'exit_date': timestamp,
                    'exit_price': exit_price,
                    'profit': profit # Safely get bar count
                }
                final_trade_log_list.append(trade_details)
                # --- END OF ADDED SECTION ---

                print(f"  -> EOD EXIT for {leg_name}. Profit: ${profit:.2f}")
                del open_positions[leg_name]




    # --- Part B: Check for NEW Hedge Entries based on the signal ---
    hedge_signal = row['hedge']
    #print(f"hedge signal is {hedge_signal}")
    
    if pd.notna(hedge_signal) and hedge_signal < 0 and timestamp.time() < time(14, 15) and timestamp.time() > time(9, 30):
        qty_to_add = abs(hedge_signal) * 100
        print(f"Hedge activated on {timestamp} and qty is {qty_to_add}")
        # 1. First, find the ATM strike from the current row's SPY price
        current_spy_price = row['SPY_close']
        if pd.isna(current_spy_price): continue
        
        atm_strike = round_to_nearest(current_spy_price, 1)+1
        expiry_str = timestamp.strftime('%Y-%m-%d')

        # 2. Now, call your function with the specific ATM strike
        atm_call_data = fetch_single_option_at_timestamp(
            con=con, ticker='SPY', timestamp=timestamp,
            strike=atm_strike,
            expiry_str=expiry_str,
            option_type=1,  DTE=1  )
        #print(f"ATM call is {atm_call_data}")
        
        if atm_call_data is not None and len(atm_call_data)>=1:
             #1. Extract the scalar values
            strike_val = atm_call_data['strike'].iloc[0]
            bid_price_val = atm_call_data['bidPrice'].iloc[0]
                # 2. Get the expiry date value AND THEN convert and format it
            expiry_val = atm_call_data['expiry'].iloc[0]
            expiry_date_obj = pd.to_datetime(expiry_val)
            
            # Format for the leg name (YYYYMMDD)
            expiry_for_leg_name = expiry_date_obj.strftime('%Y%m%d')
            # Format for storing and querying (YYYY-MM-DD)
            expiry_for_storage = expiry_date_obj.strftime('%Y-%m-%d')

            # 3. Construct the leg name
            leg_name = f"C_{strike_val}_{expiry_for_leg_name}"
            
            
            if leg_name not in open_positions:
                open_positions[leg_name] = {
                    'entry_price': bid_price_val,
                    'qty': 0,
                    'strike': strike_val,
                    'expiry': expiry_str, # Store the 'YYYY-MM-DD' version
                    'entry_timestamp': timestamp
                }
            
            open_positions[leg_name]['qty'] += qty_to_add
            print(f"  -> HEDGE ENTRY/ADD for {leg_name}: Selling {qty_to_add} contracts.")
            print(f"Strike is {strike_val}")
# --- Convert the final log to a DataFrame after the loop ---
if final_trade_log_list:
    local_trade_log2 = pd.DataFrame(final_trade_log_list)
else:
    print("No trades were made.")

🚀 Starting backtest for 2342 timestamps...
Hedge activated on 2024-01-18 11:30:00 and qty is 600.0
  -> HEDGE ENTRY/ADD for C_476.0_20240118: Selling 600.0 contracts.
Strike is 476.0
Hedge activated on 2024-01-18 12:00:00 and qty is 1900.0
  -> HEDGE ENTRY/ADD for C_475.0_20240118: Selling 1900.0 contracts.
Strike is 475.0
Hedge activated on 2024-01-18 12:45:00 and qty is 500.0
  -> HEDGE ENTRY/ADD for C_474.0_20240118: Selling 500.0 contracts.
Strike is 474.0
Exit called at 2024-01-18 15:45:00 and entry was 0.24 and exit is 0.51
  -> EOD EXIT for C_476.0_20240118. Profit: $-162.00
Exit called at 2024-01-18 15:45:00 and entry was 0.46 and exit is 1.38
  -> EOD EXIT for C_475.0_20240118. Profit: $-1748.00
Exit called at 2024-01-18 15:45:00 and entry was 0.46 and exit is 2.31
  -> EOD EXIT for C_474.0_20240118. Profit: $-925.00
Hedge activated on 2024-01-19 10:00:00 and qty is 900.0
  -> HEDGE ENTRY/ADD for C_479.0_20240119: Selling 900.0 contracts.
Strike is 479.0
Hedge activated on 202

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  -> HEDGE ENTRY/ADD for C_559.0_20240822: Selling 300.0 contracts.
Strike is 559.0
Hedge activated on 2024-08-22 12:15:00 and qty is 600.0
  -> HEDGE ENTRY/ADD for C_559.0_20240822: Selling 600.0 contracts.
Strike is 559.0
Hedge activated on 2024-08-22 12:30:00 and qty is 100.0
  -> HEDGE ENTRY/ADD for C_561.0_20240822: Selling 100.0 contracts.
Strike is 561.0
Hedge activated on 2024-08-22 13:00:00 and qty is 100.0
  -> HEDGE ENTRY/ADD for C_559.0_20240822: Selling 100.0 contracts.
Strike is 559.0
Hedge activated on 2024-08-22 13:15:00 and qty is 100.0
  -> HEDGE ENTRY/ADD for C_558.0_20240822: Selling 100.0 contracts.
Strike is 558.0
Exit called at 2024-08-22 15:45:00 and entry was 0.69 and exit is 0.02
  -> EOD EXIT for C_559.0_20240822. Profit: $938.00
Exit called at 2024-08-22 15:45:00 and entry was 0.43 and exit is 0.0
  -> EOD EXIT for C_561.0_20240822. Profit: $43.00
Exit called at 2024-08-22 15:45:00 and entry was 0.75 and exit is 0.03
  -> EOD EXIT for C_558.0_20240822. Profi

In [1]:
tt = pd.DataFrame(final_trade_log_list)

NameError: name 'pd' is not defined

In [15]:
tt

,name,Side,entry_date,entry_price,qty,exit_date,exit_price,profit
0,C_476.0_20240118,SELL,2024-01-18 11:30:00,0.24000000,600.00000000,2024-01-18 15:45:00,0.51000000,-162.00000000
1,C_475.0_20240118,SELL,2024-01-18 12:00:00,0.46000000,1900.00000000,2024-01-18 15:45:00,1.38000000,-1748.00000000
2,C_474.0_20240118,SELL,2024-01-18 12:45:00,0.46000000,500.00000000,2024-01-18 15:45:00,2.31000000,-925.00000000
3,C_479.0_20240119,SELL,2024-01-19 10:00:00,0.58000000,1400.00000000,2024-01-19 15:45:00,2.93000000,-3290.00000000
4,C_480.0_20240119,SELL,2024-01-19 11:30:00,0.34000000,600.00000000,2024-01-19 15:45:00,2.00000000,-996.00000000
...,...,...,...,...,...,...,...,...
175,C_560.0_20240830,SELL,2024-08-30 11:45:00,0.70000000,1500.00000000,2024-08-30 15:45:00,1.73000000,-1545.00000000
176,C_559.0_20240830,SELL,2024-08-30 12:30:00,0.72000000,300.00000000,2024-08-30 15:45:00,2.62000000,-570.00000000
177,C_558.0_20240903,SELL,2024-09-03 10:30:00,0.98000000,200.00000000,2024-09-03 15:45:00,0.00000000,196.00000000
178,C_557.0_20240903,SELL,2024-09-03 10:45:00,1.01000000,400.00000000,2024-09-03 15:45:00,0.00000000,404.00000000


In [17]:
tt['profit']

0      -162.00000000
1     -1748.00000000
2      -925.00000000
3     -3290.00000000
4      -996.00000000
           ...      
175   -1545.00000000
176    -570.00000000
177     196.00000000
178     404.00000000
179     216.00000000
Name: profit, Length: 180, dtype: float64